In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from datetime import datetime
from pytz import timezone

In [19]:
#tratar formato da data para o continente e pais de origem, e como a base do CS
fuso = 'America/Sao_Paulo'
formato_data = '%Y-%m-%d'
data_corrente = datetime.now(timezone(fuso)).strftime(formato_data)

In [20]:
##deixando a captura do arquivo de acordo com a data corrente
arquivo_entrada = spark.read.json('gs://base_voos_laize/dados_brutos/'+data_corrente+'.json') 
arquivo_entrada.createOrReplaceTempView('TB_VOO') #CRIANDO UM DF SPARK

In [9]:
query = """
SELECT *,
CASE 
     WHEN DISTANCE BETWEEN 0 AND 1000 THEN 1
     WHEN DISTANCE BETWEEN 1001 AND 2000 THEN 2
     WHEN DISTANCE BETWEEN 2001 AND 3000 THEN 3
     WHEN DISTANCE BETWEEN 3001 AND 4000 THEN 4
     WHEN DISTANCE BETWEEN 4001 AND 5000 THEN 5
END CATEGORIA_DISTANCIA
FROM TB_VOO LIMIT 100
"""

In [11]:
query_saida = spark.sql(query)

In [21]:
##criando um arquivo de saida dentro do GCP e uma sub pasta de acordo com a data corrente

storage = 'gs://base_voos_laize/dados_tratados/'+data_corrente+'_ETL_VOOS'

In [22]:
##Instrução que vai gerar o arquivo no storage acima
##coalesce - redução de particoes, melhora de desempenho 

query_saida.coalesce(1).write.format('json').save(storage)

25/04/05 04:49:07 WARN YarnAllocator: Container from a bad node: container_1743825045034_0001_01_000004 on host: cluster-7cdd-m.us-central1-f.c.etl-gcp-455901.internal. Exit status: 143. Diagnostics: [2025-04-05 04:49:06.479]Container killed on request. Exit code is 143
[2025-04-05 04:49:06.480]Container exited with a non-zero exit code 143. 
[2025-04-05 04:49:06.493]Killed by external signal
.
25/04/05 04:49:07 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 4 for reason Container from a bad node: container_1743825045034_0001_01_000004 on host: cluster-7cdd-m.us-central1-f.c.etl-gcp-455901.internal. Exit status: 143. Diagnostics: [2025-04-05 04:49:06.479]Container killed on request. Exit code is 143
[2025-04-05 04:49:06.480]Container exited with a non-zero exit code 143. 
[2025-04-05 04:49:06.493]Killed by external signal
.
25/04/05 04:49:07 ERROR YarnScheduler: Lost executor 4 on cluster-7cdd-m.us-central1-f.c.etl-gcp-455901.internal: Container f